In [ ]:
from scipy import stats
import scipy.special as special
from scipy.optimize import curve_fit
from ipywidgets import interact
import ipywidgets as widgets
import hyperspy.api as hys
import numpy as np
import matplotlib.pyplot as plt
import tkinter.filedialog as tkf

In [ ]:
%matplotlib inline

In [ ]:
ll_adr = tkf.askopenfilename()
print(ll_adr)

In [ ]:
ll = hys.load(ll_adr, signal_type="EELS")
print(ll)

In [ ]:
zlp_shift = ll.estimate_zero_loss_peak_centre().data
print(zlp_shift.shape)
spike = np.where(zlp_shift > 10)
print(spike)
for i in range(len(spike[0])):
    iy, ix = spike[0][i], spike[1][i]
    ll.data[iy, ix] = ll.data[iy-1, ix]
    print("the spectrum at %d column, %d row will be replaced by the spectrum at %d column, %d row"%(iy, ix, iy-1, ix))

In [ ]:
ll.align_zero_loss_peak()
print(ll)

In [ ]:
zlp_shift_after = ll.estimate_zero_loss_peak_centre().data
print(np.unique(zlp_shift_after))

In [ ]:
def beta_function(a, b):
    return special.gamma(a) * special.gamma(b) / special.gamma(a*b)

def pearson7(x, nor, alpha, lmb, m, cor_term):
    return nor*np.power((1+np.power(((x-lmb)/alpha), 2)), -m) / (alpha * beta_function(m-0.5, 0.5)) + cor_term

In [ ]:
# for a quick test
%matplotlib qt
step = ll.axes_manager[2].scale
left = ll.axes_manager[2].offset + step
e_size = ll.axes_manager[2].size
print(left, step, e_size)
x_range = np.arange(left, (e_size-1)*step+left, step)
print(x_range[0], x_range[-1])
print(x_range.shape)

plt.figure(figsize=(10,10))

i = 0
j = 1
ll_zlp_removed = ll.isig[left:]
print(ll_zlp_removed.axes_manager[2].offset, ll_zlp_removed.axes_manager[2].scale)
temp = ll_zlp_removed.data[i, j].copy()
print(temp.shape)
plt.plot(temp, label="original spectrum")
maxid = np.argmax(temp)
print(maxid)
left_side = temp[:(maxid+1)]
zlp_mirror = np.zeros(maxid*2+1)
zlp_mirror[:(maxid+1)] = left_side
zlp_mirror[maxid:] = np.flip(left_side)
print(zlp_mirror.shape)
plt.plot(zlp_mirror, label="mirrored ZLP")
e_range = np.arange(left, len(zlp_mirror)*step+left, step)
popt_o, pcov_o = curve_fit(pearson7, e_range, temp[:len(zlp_mirror)])
fitted_o = pearson7(x_range, *popt)
#plt.plot(fitted_o, label="fitted from the original ZLP")
#plt.plot(ll_zlp_removed.data[i, j] - fitted_o, label="ZLP subtracted spectrum (original ZLP fitting)")
temp[:len(zlp_mirror)] -= zlp_mirror
#plt.plot(temp, label="ZLP subtracted spectrum (zlp mirror)")
popt, pcov = curve_fit(pearson7, e_range, zlp_mirror)
fitted = pearson7(x_range, *popt)
#plt.plot(fitted, label="fitted from the mirrored ZLP")
#plt.plot(ll_zlp_removed.data[i, j] - fitted, label="ZLP subtracted spectrum (mirrored ZLP fitting)")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# set the energy range for zlp fitting
step = ll.axes_manager[2].scale
left = ll.axes_manager[2].offset + step
e_size = ll.axes_manager[2].size
print(left, step, e_size)
x_range = np.arange(left, (e_size-1)*step+left, step)
print(x_range[0], x_range[-1])
print(x_range.shape)

In [ ]:
# zlp fitting to Pearson 7 distribution (reflected zlp)
ll_zlp_removed = ll.isig[left:]

for i in range(ll.data.shape[0]):
    for j in range(ll.data.shape[1]):
        temp = ll_zlp_removed.data[i, j].copy()
        maxid = np.argmax(temp)
        left_side = temp[:(maxid+1)]
        zlp_mirror = np.zeros(maxid*2+1)
        zlp_mirror[:(maxid+1)] = left_side
        zlp_mirror[maxid:] = np.flip(left_side)
        e_range = np.arange(left, len(zlp_mirror)*step+left, step)
        try:
            popt, pcov = curve_fit(pearson7, e_range, zlp_mirror)
            ll_zlp_removed.data[i, j] -= pearson7(x_range, *popt)
        except RuntimeError:
            print("the spectrum at %d column, %d row will be replaced by the previous spectrum."%(i, j))
            if j != 0:
                ll_zlp_removed.data[i, j] = ll_zlp_removed.data[i, j-1].copy()
            elif i != 0:
                ll_zlp_removed.data[i, j] = ll_zlp_removed.data[i-1, ll.data.shape[1]-1]
            else:
                zl_zlp_removed.data[i, j] = 0
                print("unfortunately, an error occurs at the beginning, thus the first spectrum will be 0.")

In [ ]:
%matplotlib qt
ll_zlp_removed.plot()

In [ ]:
ll_zlp_removed = ll_zlp_removed.isig[1.0:5.0]
print(ll_zlp_removed)

In [ ]:
ll_zlp_removed.data += -np.min(ll_zlp_removed.data)

In [ ]:
ll_zlp_removed.remove_background(zero_fill=True, fast=False)

In [ ]:
ll_dm = hys.signals.Signal2D(np.rollaxis(np.rollaxis(ll_zlp_removed.data, 0, 3), 0, 3))
print(ll_dm)
ll_zlp_removed.save(ll_adr[:-4]+"_zlp_mirror_pearson7_fitting.hdf5")
ll_dm.save(ll_adr[:-4]+"_zlp_mirror_pearson7_fitting_dm_import.hdf5")

In [ ]:
# zlp fitting to Pearson 7 distribution
ll_zlp_removed = ll.isig[left:]
print(left, -left+step)
e_range = np.arange(left, -left+step, step)
len_zlp = len(e_range)
print(e_range.shape)

for i in range(ll.data.shape[0]):
    for j in range(ll.data.shape[1]):
        temp = ll_zlp_removed.data[i, j].copy()
        try:
            popt, pcov = curve_fit(pearson7, e_range, temp[:len_zlp])
            ll_zlp_removed.data[i, j] -= pearson7(x_range, *popt)
        except RuntimeError:
            print("the spectrum at %d column, %d row will be replaced by the previous spectrum."%(i, j))
            if j != 0:
                ll_zlp_removed.data[i, j] = ll_zlp_removed.data[i, j-1].copy()
            elif i != 0:
                ll_zlp_removed.data[i, j] = ll_zlp_removed.data[i-1, ll.data.shape[1]-1]
            else:
                ll_zlp_removed.data[i, j] = 0
                print("unfortunately, an error occurs at the beginning, thus the first spectrum will be 0.")

In [ ]:
ll_zlp_removed = ll_zlp_removed.isig[1.0:5.0]
print(ll_zlp_removed)

In [ ]:
%matplotlib qt
ll_zlp_removed.plot()

In [ ]:
ll_zlp_removed.data[np.where(ll_zlp_removed < 0)] = 0

In [ ]:
ll_dm = hys.signals.Signal2D(np.rollaxis(np.rollaxis(ll_zlp_removed.data, 0, 3), 0, 3))
print(ll_dm)
ll_zlp_removed.save(ll_adr[:-4]+"_pearson7_fitting.hdf5")
ll_dm.save(ll_adr[:-4]+"_pearson7_fitting_dm_import.hdf5")

In [ ]:
# remove zero loss peak (reflected tail)
zl_removed = []
limit_left = 100
limit_right = 1000
for i in range(ll.data.shape[0]):
    for j in range(ll.data.shape[1]):
        temp = ll.data[i, j]
        maxid = np.argmax(temp)
        reverse = np.flip(temp[:(maxid+1)])
        temp = temp[maxid:]
        temp[:len(reverse)] -= reverse
        zl_removed.append(temp[limit_left:limit_right])

zl_removed = np.asarray(zl_removed).reshape(ll.data.shape[0], ll.data.shape[1], limit_right-limit_left)
zl_removed = hys.signals.Signal1D(zl_removed, signal_type="EELS")
print(zl_removed)

In [ ]:
zl_removed.metadata = ll.metadata
zl_removed.axes_manager = ll.axes_manager
zl_removed.axes_manager[2].size = limit_right - limit_left
zl_removed.axes_manager[2].offset = zl_removed.axes_manager[2].scale * (limit_left-1)
print(zl_removed.axes_manager)

In [ ]:
%matplotlib qt
zl_removed.plot()

In [ ]:
ll_dm = hys.signals.Signal2D(np.rollaxis(np.rollaxis(zl_removed.data, 0, 3), 0, 3))
print(ll_dm)
ll.save(ll_adr[:-4]+"_alined.hdf5")
ll_dm.save(ll_adr[:-4]+"_alined_dm_import.hdf5")

In [ ]:
l1 = 450.0
r1 = 480.0
hl_crop_1 = hl.isig[l1:r1]
hl_crop_1.save(hl_adr[:-4]+"_cropped_TiL23.hdf5")
hl_crop_1_dm = hys.signals.Signal2D(np.rollaxis(np.rollaxis(hl_crop_1.data, 0, 3), 0, 3))
hl_crop_1_dm.save(hl_adr[:-4]+"_cropped_TiL23_dm_import.hdf5")